In [1]:
! pip install bs4
! pip install prettytable
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib
import re
from prettytable import PrettyTable

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
URL = "https://www.indeed.com/jobs?q=data+science&start=10"
page = requests.get(URL)
soup = BeautifulSoup(page.text, "html.parser", from_encoding="utf-8")
print(soup.prettify())

/Users/paulmathai/.virtualenvs/keras_tf/lib/python3.6/site-packages/bs4/__init__.py:191: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="/s/077d714/en_US.js" type="text/javascript">
  </script>
  <script>
   window.indeed=window.indeed||{};indeed.crashtext=indeed.crashtext||{};indeed.crashtext.populate=indeed.crashtext.populate||function(c,g){var d=window[g];if(d){var e=d[""]||{},h=e.salt;if(e.hasOwnProperty("salt"))for(var b in c){for(var a=void 0,a=h,f=b.length;f;)a=33*a^b.charCodeAt(--f);a>>>=0;e.hasOwnProperty("id_length")&&(a=String(a).substring(0,e.id_length));d[a]=c[b]}else for(b in c)d[b]=[null].concat(c[b])}};indeed.crashtext.populate({"indeedapply_serp_label":["Express apply"]}, 'indeed.i18n.localeData')
  </script>
  <link href="/s/b066150/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+science" rel="alternate" title="Data Science Jobs, Employment" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+science" media="on

In [3]:
#Extracting job title.
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
      for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
          jobs.append(a["title"])
    return(jobs)
title = extract_job_title_from_result(soup)
title

['Jr. Machine Learning/Data science',
 'Statistics and Data Science Engineer',
 'Data Scientist',
 'Data Science Engineer',
 'Product Manager - Spatial Analytics & Data Science',
 'Intern - Data Science (Summer 2020)',
 'Data Scientist',
 'Data Scientist',
 'Sr Data Scientist',
 'Data Science Intern',
 'Democratization Data Scientist',
 'Data Scientist',
 'Data Management Analyst',
 'Health Data Scientist',
 'Sr Data Scientist',
 'Data Scientist',
 'Data Scientist',
 '2020 Artificial Intelligence / Machine Learning Intern',
 'Data Scientist']

In [4]:
#Extract Companies looking for employees.
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
           for b in company:
               companies.append(b.text.strip())
        else:
           sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
           for span in sec_try:
               companies.append(span.text.strip())
    return(companies)
 
company = extract_company_from_result(soup)
company



['People tech group',
 'Canary Systems Inc.',
 'Aegis Premier Technologies',
 'SteepRock, Inc.',
 'Esri',
 'MedPro Group',
 'GEICO',
 'Ford Motor Company',
 'Progrexion',
 'LexisNexis SA',
 'Ford Motor Company',
 'Best Buy',
 'FCA',
 'Genospace',
 'Progrexion Holdings Inc',
 'Flexera Software, Inc.',
 'Computer Enterprises, Inc. (CEI)',
 'Miso',
 'BankUnited']

In [5]:
#Extraction Location.
def extract_location_from_result(soup): 
    locations = []
    spans = soup.find_all("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)
location = extract_location_from_result(soup)
location


['Dearborn, MI',
 'Salt Lake City, UT',
 'Raleigh, NC',
 'Dearborn, MI 48126',
 'Richfield, MN 55423',
 'Detroit, MI 48226 (Downtown area)',
 'Boston, MA 02108 (Back Bay-Beacon Hill area)',
 'Salt Lake City, UT',
 'Itasca, IL',
 'Pittsburgh, PA']

In [6]:
#Extract Salary.
def extract_salary_from_result(soup): 
    salary = []
    spans = soup.find_all("span", attrs={"class": "salaryText"})
    for span in spans:
        salary.append(span.text)
    return(salary)
salary = extract_salary_from_result(soup)
salary

['\n$110,000 - $140,000 a year', '\n$60 - $70 an hour']

In [7]:
#Extract Rating.
def extract_ratingsContent_from_result(soup): 
    ratingsContent = []
    spans = soup.find_all('span', attrs={'class': 'ratingsContent'})
    for span in spans:
        try:
            ratingsContent.append(span.text)
        except:
            ratingsContent.append("Nothing_found")
    return(ratingsContent)

ratingsContent = extract_ratingsContent_from_result(soup)
ratingsContent


['\n3.8\n\n\n\n\n',
 '\n4.0\n\n\n\n\n',
 '\n3.8\n\n\n\n\n',
 '\n3.2\n\n\n\n\n',
 '\n3.5\n\n\n\n\n',
 '\n4.2\n\n\n\n\n',
 '\n3.4\n\n\n\n\n',
 '\n3.7\n\n\n\n\n',
 '\n4.2\n\n\n\n\n',
 '\n3.9\n\n\n\n\n',
 '\n4.0\n\n\n\n\n',
 '\n4.5\n\n\n\n\n',
 '\n3.4\n\n\n\n\n',
 '\n3.6\n\n\n\n\n',
 '\n3.6\n\n\n\n\n',
 '\n3.8\n\n\n\n\n']

In [8]:
#Extract Summary.
def extract_summary_from_result(soup): 
  summaries = []
  spans = soup.find_all("div", attrs={"class": "summary"})
  for span in spans:
    summaries.append(span.text.strip())
  return(summaries)
  summaries
extract_summary_from_result(soup)

['Machine Learning/ Data Science Engineer*.Working closely with the current team of Data scientists and engineers on various complex problems.',
 'We are looking for an individual with a statistical and data science background to assist supporting the implementation of our systems at various locations.',
 'The Data Scientist will help us design, create and deliver products driven from vast consumer data resources.Data mining using state-of-the-art methods.',
 'Life sciences technology and services company with a configurable web-based data management platform used by leading pharmaceutical companies seeks a seasoned…',
 'Leverage your discerning mindset to separate data science market trends from science fiction and media hype.Experience with data science, business intelligence…',
 'Active pursuit of Data Science, Computer Science, Mathematics or related degree with at least two years completed (and one year remaining).',
 'We see our projects through the entire data science lifecycle,

<hr>
Original
<hr>

In [14]:
columns = ["[job_title]", "[company_name]", "[location]", "[salary]", "[ratingsContent]", "[summary]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science' + '&start=' + (str(start)*10))
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll('span', attrs={'class': 'salaryText'})
        if(len(d)==0):
          job_post.append("NA")
        for span in d:
              job_post.append(span.text)
        e = div.findAll('span', attrs={'class': 'ratingsContent'})
        if(len(e)==0):
          job_post.append("NA")
        for span in e:
            job_post.append(span.text)
        f = div.findAll("div", attrs={"class": "summary"})
        if(len(f)==0):
            job_post.append("NA")
        for span in f:
            job_post.append(span.text)

            sample_df.loc[num]=job_post

sample_df.to_csv("data.csv", encoding='utf-8')

<hr>
Entry Level
<hr>

In [59]:
columns = ["[job_title]","[location]","[company]", "[entry_level]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&explvl=entry_level' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("entry_level.csv", encoding='utf-8')

<hr>
Mid Level
<hr>

In [64]:
columns = ["[job_title]","[location]","[company]", "[mid_level]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 100):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&explvl=mid_level' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("mid_level.csv", encoding='utf-8')

<hr>
Senior Level
<hr>


In [65]:
columns = ["[job_title]","[location]","[company]", "[senior_level]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&explvl=senior_level' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("senior_level.csv", encoding='utf-8')

<hr>
Full Time
<hr>

In [66]:
columns = ["[job_title]","[location]","[company]", "[fulltime]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&jt=fulltime' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("fulltime.csv", encoding='utf-8')

<hr>
Contract
<hr>

In [67]:
columns = ["[job_title]","[location]","[company]", "[contract]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&jt=contract' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("contract.csv", encoding='utf-8')

<hr>
Internship
<hr>

In [68]:
columns = ["[job_title]","[location]","[company]", "[internship]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&jt=internship' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("internship.csv", encoding='utf-8')

<hr>
Part-Time
<hr>

In [13]:
columns = ["[job_title]", "[parttime]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 500):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&jt=parttime' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("parttime.csv", encoding='utf-8')

<hr>
Temporary
<hr>

In [ ]:
columns = ["[job_title]","[location]","[company]", "[temporary]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&jt=temporary' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("temporary.csv", encoding='utf-8')

<hr>
Commission
<hr>

In [ ]:
columns = ["[job_title]","[location]","[company]", "[commission]"]

sample_df = pd.DataFrame(columns=columns)

for start in range(0, 1000):
    page = requests.get('http://www.indeed.com/jobs?q=data+science&jt=commission' + '&start=' + (str(start)*10))
    time.sleep(1)
    soup = BeautifulSoup(page.text, "lxml")
    for div in soup.find_all(name="div", attrs={"class":"row"}): 

        num = (len(sample_df) + 1)
        job_post = [] 
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"]) 
        company = div.find_all(name="span", attrs={"class":"company"}) 
        if len(company) > 0: 
            for b in company:
                job_post.append(b.text.strip()) 
        else: 
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text)
        c = div.findAll("span", attrs={"class": "location"})
        if(len(c)==0):
          job_post.append("NA") 
        for span in c: 
            job_post.append(span.text)
        d = div.findAll("span", attrs={"class": "location"})
        for span in d: 
            job_post.append("yes") 
            sample_df.loc[num]=job_post

sample_df.to_csv("commission.csv", encoding='utf-8')